# Chiller plant data cleaning 

The purpose of this notebook is to outline methods for determining which chiller data sets should be used in the regression analysis.

Components:
 - remove data points that contain NaN values
 - remove data points where OptimumControl = 0
 - remove data points with sensor alarms

## Imports

In [1]:
import pandas as pd
import numpy as np
import glob
import os
import bas_filter

## Reading Data into Jupyter

In [2]:
df = pd.read_csv('../../Capstone/Files/Plt1 h 2017-09.csv')
key = pd.read_excel('../../Capstone/Files/Plt1 Points List.xlsx')

### Test to drop NaN values from data

In [3]:
df.dropna()

,timestamp,CH3LoadDisp,CH3PctLoad,CH4LoadDisp,CH4PctLoad,CH5LoadDisp,CH5PctLoad,CHkW%,CHTON%,PercentAnnualSavings,...,CH5SURGE,LOOPREQ,RUNCHLR3,RUNCHLR4,RUNCHLR5,CH3SURGECNT,CH4SURGECNT,CH5SURGECNT,CLGMODE,StationSTG
60,2017-09-03T12:00:00.000Z,0.000000,0.00000,87.101169,87.101168,0.982292,0.982292,71.551070,86.750559,0.000124,...,0.0,1.0,0.000000,1.000000,0.166667,13.0,0.0,0.0,0.0,1.166667
61,2017-09-03T13:00:00.000Z,0.000000,0.00000,61.628025,61.649601,32.699242,32.699242,28.800355,46.600451,0.000184,...,0.0,1.0,0.000000,1.000000,1.000000,13.0,0.0,0.0,0.0,2.000000
62,2017-09-03T14:00:00.000Z,0.000000,0.00000,65.271903,65.271902,34.757415,34.667396,31.222250,49.421877,0.000195,...,0.0,1.0,0.000000,1.000000,1.000000,13.0,0.0,0.0,0.0,2.000000
63,2017-09-03T15:00:00.000Z,0.000000,0.00000,62.073736,60.817505,35.873339,35.490721,31.827724,50.010946,0.000182,...,0.0,1.0,0.166667,1.000000,1.000000,13.0,0.0,0.0,0.0,2.000000
66,2017-09-03T18:00:00.000Z,0.000000,0.00000,66.073464,66.073463,37.944247,37.944247,34.189070,52.350163,0.000190,...,0.0,1.0,0.000000,1.000000,1.000000,13.0,0.0,0.0,0.0,2.000000
67,2017-09-03T19:00:00.000Z,0.000000,0.00000,64.536953,64.555182,36.308257,36.305479,32.816663,50.486619,0.000187,...,0.0,1.0,0.000000,1.000000,1.000000,13.0,0.0,0.0,0.0,2.000000
68,2017-09-03T20:00:00.000Z,0.000000,0.00000,60.918676,60.918676,33.823301,33.823301,30.818600,47.683522,0.000179,...,0.0,1.0,0.000000,1.000000,1.000000,13.0,0.0,0.0,0.0,2.000000
69,2017-09-03T21:00:00.000Z,0.000000,0.00000,57.567634,57.580631,32.202592,32.206610,29.244369,44.708515,0.000156,...,0.0,1.0,0.000000,1.000000,1.000000,13.0,0.0,0.0,0.0,2.000000
70,2017-09-03T22:00:00.000Z,0.000000,0.00000,40.959836,41.008944,42.549615,42.540327,31.184866,50.812883,0.000141,...,0.0,1.0,0.000000,0.666667,1.000000,13.0,0.0,0.0,0.0,1.583333
71,2017-09-03T23:00:00.000Z,0.000000,0.00000,0.000000,0.000000,78.306298,77.869101,50.595238,78.069033,0.000098,...,0.0,1.0,0.000000,0.000000,1.000000,13.0,0.0,0.0,0.0,1.000000


Result: only 11 data points of the dataframe contained no blank cells. This can be refined by extracting the BAS parameters.

## BAS Extraction

In [4]:
# collects DataPointNames from the points list spreadsheet that are BAS point types
key_bas = key.loc[key['PointType'].str.contains("BAS")==True,'DataPointName']

#converts pandas series to a list for future use
val = key_bas.values.T.tolist()

#removes DataPointNames that containt the prefix CHWV
vals = [x for x in val if not x.startswith('CHWV')]

#tests whether all values from the point list spreadsheet are column headings of the dataset
for x in vals:
    if x not in df.columns:
        #prints and removes any string not found in the data
        print(x)
        vals.remove(x)
        
#expresses data using columns specified by the vals list
bas = df[vals+['OptimumControl', 'kW/Ton']]
print('Original data contains '+str(df.shape[0])+' points and '+str(df.shape[1])+ ' dimensions.')
print('Filtered data contains '+str(bas.dropna().shape[0])+' points and '+str(bas.dropna().shape[1])+ ' dimensions.')

CommunicationFailure_COV
Original data contains 1465 points and 414 dimensions.
Filtered data contains 1432 points and 66 dimensions.


This result tells us that very few of the BAS points are missing entries. The data can be further filtered with statements exemplified below.

In [5]:
bas[bas['OptimumControl'] == 1]



,CDWVLV,CDWVLV2,CHkW,CHWDP2,CHWFLO,CHWRT,CHWRT2,CHWRT3,CHWST,CHWST2,...,SCHWP4SPD,SCHWP4SS,SCHWP5Failed,SCHWP5HZ,SCHWP5kW,SCHWP5S,SCHWP5SPD,SCHWP5SS,OptimumControl,kW/Ton
0,1.0,1.0,107.750000,10.749720,671.000000,55.882812,52.856771,57.054688,42.005208,42.500000,...,80.666344,1.0,0.0,48.400000,2.758333,1.0,80.666344,1.0,1.0,0.476109
1,1.0,1.0,101.333333,10.560215,674.500000,55.416667,52.377604,57.437500,42.036458,42.500000,...,80.819121,1.0,0.0,48.491666,2.783333,1.0,80.819121,1.0,1.0,0.472175
2,1.0,1.0,121.500000,6.449532,621.125000,56.752604,52.755208,58.388021,41.960938,42.854167,...,99.277384,1.0,0.0,59.566667,4.533333,1.0,99.277384,1.0,1.0,0.518674
3,1.0,1.0,116.333333,8.820532,592.291667,55.851562,53.898438,58.132812,41.820312,42.671875,...,99.985714,1.0,0.0,59.991666,4.491667,1.0,99.985714,1.0,1.0,0.530859
4,1.0,1.0,116.833333,8.449738,593.541667,56.518229,53.885417,58.442708,41.869792,42.500000,...,98.832940,1.0,0.0,59.300000,4.358333,1.0,98.832940,1.0,1.0,0.521720
5,1.0,1.0,116.416667,8.006648,606.541667,56.609375,53.651042,58.708333,41.895833,42.500000,...,99.999603,1.0,0.0,60.000000,4.550000,1.0,99.999603,1.0,1.0,0.516152
6,1.0,1.0,105.416667,9.884717,573.583333,55.822917,53.562500,58.403646,41.851562,42.768229,...,97.235722,1.0,0.0,58.341666,4.108333,1.0,97.235722,1.0,1.0,0.505199
7,1.0,1.0,101.500000,10.232507,569.750000,56.296875,53.656250,58.497396,42.242188,43.265625,...,91.110744,1.0,0.0,54.666665,3.600000,1.0,91.110744,1.0,1.0,0.499915
8,1.0,1.0,115.083333,9.928898,620.166667,55.500000,53.578125,57.869792,41.596354,42.562500,...,97.555166,1.0,0.0,58.533333,4.350000,1.0,97.555166,1.0,1.0,0.491188
9,1.0,1.0,123.250000,7.364924,647.666667,55.729167,53.281250,58.164062,41.614583,42.562500,...,99.999603,1.0,0.0,60.000000,4.700000,1.0,99.999603,1.0,1.0,0.500251


## Function Workup 

In [6]:
def data_import(dat_folder, string, keys):
    
    dat_list = [f for f in glob.glob(os.path.join(dat_folder, string + '*'))]
    print(dat_list)
    
    df = pd.DataFrame()
    for lst in dat_list:
        df_add = pd.read_csv(lst)
        df = pd.concat([df, df_add], ignore_index=True)
    key = pd.read_excel(keys)
    
    return df, key

In [7]:
df, key = data_import('../../Capstone/files', 'Plt1 h', '../../Capstone/files/Plt1 Points List.xlsx')

['../../Capstone/files\\Plt1 h 2016-11.csv', '../../Capstone/files\\Plt1 h 2017-01.csv', '../../Capstone/files\\Plt1 h 2017-03.csv', '../../Capstone/files\\Plt1 h 2017-05.csv', '../../Capstone/files\\Plt1 h 2017-07.csv', '../../Capstone/files\\Plt1 h 2017-09.csv']


In [8]:
def data_BAS(df, key):
    # collects DataPointNames from the points list spreadsheet that are BAS point types
    key_bas = key.loc[key['PointType'].str.contains("BAS")==True,'DataPointName']

    #converts pandas series to a list for future use
    val = key_bas.values.T.tolist()

    #removes DataPointNames that containt the prefix CHWV
    vals = [x for x in val if not x.startswith('CHWV')]

    #tests whether all values from the point list spreadsheet are column headings of the dataset
    for x in vals:
        if x not in df.columns:
            #prints and removes any string not found in the data
            print(x)
            vals.remove(x)
        
    #expresses data using columns specified by the vals list
    bas = df[vals+['OptimumControl', 'kW/Ton']]
    
    print('Original data contains '+str(df.shape[0])+' points and '+str(df.shape[1])+ ' dimensions.')
    print('Filtered data contains '+str(bas.dropna().shape[0])+' points and '+str(bas.dropna().shape[1])+ ' dimensions.')
    return bas.dropna()

In [9]:
bas = data_BAS(df, key)
bas.shape

CommunicationFailure_COV
Original data contains 8516 points and 414 dimensions.
Filtered data contains 8115 points and 66 dimensions.


(8115, 66)

In [10]:
key_alarm = key[key['Units'].str.contains("Normal/Alarm")==True]
key_alarm_BAS = key_alarm.loc[key['PointType'].str.contains("BAS")==True, 'DataPointName']
print(key_alarm_BAS)

37    CommunicationFailure
39              CTTR_ALARM
54            PCHWP3Failed
60            PCHWP4Failed
66            PCHWP5Failed
72            SCHWP3Failed
78            SCHWP4Failed
84            SCHWP5Failed
Name: DataPointName, dtype: object


In [11]:
for alm in key_alarm_BAS:
    bas = bas[bas[alm] == 0]
bas = bas[bas['OptimumControl'] == 1]

In [12]:
bas.shape

(7952, 66)

In [13]:
def alarm_filter(bas, key):
    key_alarm = key[key['Units'].str.contains("Normal/Alarm")==True]
    key_alarm_BAS = key_alarm.loc[key['PointType'].str.contains("BAS")==True, 'DataPointName']
    
    for alm in key_alarm_BAS:
        bas = bas[bas[alm] == 0]
    bas = bas[bas['OptimumControl'] == 1]
    return bas

In [14]:
bas1 = alarm_filter(bas, key)
bas1.shape

(7952, 66)

In [15]:
bas1[bas1['HX1CDWVLV'] < 1].shape

(7952, 66)

## Function .py Testing

In [16]:
df, key = bas_filter.data_import('../../Capstone/files', 'Plt1 h', '../../Capstone/files/Plt1 Points List.xlsx')
bas = bas_filter.data_BAS(df, key)
bas1 = bas_filter.alarm_filter(bas, key)
bas1.shape


['../../Capstone/files\\Plt1 h 2016-11.csv', '../../Capstone/files\\Plt1 h 2017-01.csv', '../../Capstone/files\\Plt1 h 2017-03.csv', '../../Capstone/files\\Plt1 h 2017-05.csv', '../../Capstone/files\\Plt1 h 2017-07.csv', '../../Capstone/files\\Plt1 h 2017-09.csv']
CommunicationFailure_COV
Original data contains 8516 points and 414 dimensions.
Filtered data contains 8115 points and 66 dimensions.


(7952, 66)

In [17]:
df, key = bas_filter.import_and_filter('../../Capstone/files', 'Plt1 h', '../../Capstone/files/Plt1 Points List.xlsx')
df.shape

['../../Capstone/files\\Plt1 h 2016-11.csv', '../../Capstone/files\\Plt1 h 2017-01.csv', '../../Capstone/files\\Plt1 h 2017-03.csv', '../../Capstone/files\\Plt1 h 2017-05.csv', '../../Capstone/files\\Plt1 h 2017-07.csv', '../../Capstone/files\\Plt1 h 2017-09.csv']
CommunicationFailure_COV
Original data contains 8516 points and 414 dimensions.
Filtered data contains 8115 points and 66 dimensions.


(7952, 66)